In [1]:
import pandas as pd
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="exam_152",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

# Load data into DataFrame
query = """ 
SELECT \"date\", sum(enemy_losses_artillery) as data_field 
FROM battle_reports 
group by \"date\"
order by date;
"""
df = pd.read_sql(query, conn)

# Close connection
conn.close()


C:\Users\user11\AppData\Local\Temp\ipykernel_16276\1071584325.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
df.head()

,date,data_field
0,2023-03-30,8
1,2023-03-31,22
2,2023-04-01,27
3,2023-04-02,16
4,2023-04-03,17


In [ ]:
#%pip install tensorflow numpy pandas


  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install --upgrade numpy


  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# ----------------------------
# Step 1: Load and prepare data
# ----------------------------

# Load your dataframe
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')
df.set_index('date', inplace=True)

data = df[['data_field']].values  # shape: (n_samples, 1)

# Normalize the data (LSTM works better with scaled data)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# ----------------------------
# Step 2: Create sequences
# ----------------------------

def create_sequences(data, look_back=7):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 7
X, y = create_sequences(data_scaled, look_back)

# Reshape X for LSTM: (samples, timesteps, features)
print("X shape:", X.shape)  # e.g. (samples, 7, 1)

# ----------------------------
# Step 3: Train/Test Split
# ----------------------------

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# ----------------------------
# Step 4: Build and train LSTM model
# ----------------------------

model = Sequential([
    LSTM(50, activation='relu', input_shape=(look_back, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

# ----------------------------
# Step 5: Evaluate and save
# ----------------------------

loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")



X shape: (724, 7, 1)
Epoch 1/50


c:\Users\user11\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1899
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0437
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0435
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0391
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0372
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0394
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0408
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0389
Epoch 9/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0420
Epoch 10/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0400
Epoch 11/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0409
Epoch 12/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0364
Epoch 13/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0392
Epoch 14/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0387
Epoch 15/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0368
Epoch 16/50
37/37 ━━━━━━━━━━━━

In [8]:
import json
# Get the last known window
last_window = data_scaled[-look_back:].reshape(1, look_back, 1)

future_preds = []
current_input = last_window.copy()

for _ in range(7):
    pred = model.predict(current_input)[0]
    future_preds.append(pred[0])

    # Ensure pred is reshaped correctly to match (1, 1, 1)
    pred_reshaped = pred.reshape(1, 1, 1)

    # Concatenate along the time-step axis (axis=1)
    current_input = np.concatenate([current_input[:, 1:, :], pred_reshaped], axis=1)


# Inverse transform to original scale
future_preds_original = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))

# Build forecast with dates
last_date = df.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7)
forecast_df = pd.DataFrame({'date': forecast_dates, 'forecast': future_preds_original.flatten()})
print(forecast_df)

# Save to JSON
forecast_dict = dict(zip(forecast_df['date'].astype(str), forecast_df['forecast'].round(2)))
with open('forecast.json', 'w') as f:
    json.dump(forecast_dict, f, indent=4)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
        date   forecast
0 2025-03-30  25.041058
1 2025-03-31  25.329384
2 2025-04-01  25.164251
3 2025-04-02  25.361736
4 2025-04-03  25.678030
5 2025-04-04  25.010176
6 2025-04-05  24.902107
